In [0]:
pip install --no-index scikit-learn matplotlib tensorflow_gpu pandas numpy spacy

ERROR: Could not find a version that satisfies the requirement tensorflow_gpu (from versions: none)
ERROR: No matching distribution found for tensorflow_gpu


In [0]:
%tensorflow_version 2.x

In [68]:
pip install sacrebleu

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 17.1MB 245kB/s 


In [67]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import pickle
from functools import partial
import tensorflow as tf
import spacy
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import os
import io
import time

ModuleNotFoundError: ignored

In [4]:
print(tf.__version__)

2.2.0-rc2


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append("..") # Require to have the utilities packages in path

In [8]:
%cd '/content/drive/My Drive/Colab Notebooks/IFT 6759/Project 2'

/content/drive/My Drive/Colab Notebooks/IFT 6759/Project 2


In [64]:
!ls

checkpoints			   Transformer.ipynb	  valid_Target.txt
Data				   valid_en_dataset.txt
EDA_SeqSeq_ift6759-project2.ipynb  valid_predictions.txt


In [10]:
data_path = Path(r"/content/drive/My Drive/Colab Notebooks/IFT 6759/Project 2/Data/")
files = list(data_path.glob("*"))
files

[PosixPath('/content/drive/My Drive/Colab Notebooks/IFT 6759/Project 2/Data/train.lang1'),
 PosixPath('/content/drive/My Drive/Colab Notebooks/IFT 6759/Project 2/Data/train.lang2'),
 PosixPath('/content/drive/My Drive/Colab Notebooks/IFT 6759/Project 2/Data/unaligned.en'),
 PosixPath('/content/drive/My Drive/Colab Notebooks/IFT 6759/Project 2/Data/unaligned.fr')]

In [0]:
with open(data_path / "unaligned.en", 'r') as f:
    unaligned_en = [line.rstrip() for line in f] # Remove the \n
    unaligned_en = pd.DataFrame(unaligned_en, columns=["text"])
    f.close()
    
with open(data_path / "unaligned.fr", 'r') as f:
    unaligned_fr = [line.rstrip() for line in f] # Remove the \n
    unaligned_fr = pd.DataFrame(unaligned_fr, columns=["text"])
    f.close()

with open(data_path / "train.lang1", 'r') as f:
    train_lang1_en = [line.rstrip() for line in f] # Remove the \n
    train_lang1_en = pd.DataFrame(train_lang1_en, columns=["text"])
    f.close()

with open(data_path / "train.lang2", 'r') as f:
    train_lang2_fr = [line.rstrip() for line in f] # Remove the \n
    train_lang2_fr = pd.DataFrame(train_lang2_fr, columns=["text"])
    f.close()

In [12]:
max = 0
countEntry = 0
for index, row in train_lang2_fr.iterrows():
    count = row["text"].count('. ')
    
    if count > 1:
        countEntry += 1
    if count > max:
        max = count
        sentence = row["text"]

print(countEntry, "entries have more than 1 sentence", '\n')
print('One entry has at least', max, 'sentences:')
print('"', sentence, '"')

68 entries have more than 1 sentence 

One entry has at least 5 sentences:
" Nous allons devoir réapprendre ce que signifie la construction de toutes ces choses . La rue . Le pâté de maisons . Comment composer un espace public qui soit à la fois grand et petit ? La cour . La place publique . Et comment mettre en valeur cet immobilier . "


In [0]:
# Based on: https://www.tensorflow.org/tutorials/text/nmt_with_attention
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w



In [0]:
def create_dataset():
    frText = []
    enText = []
    for ((indexFr, rowFr), (indexEn, rowEn))  in zip(train_lang2_fr.iterrows(), train_lang1_en.iterrows()):
        frText.append(preprocess_sentence(rowFr['text']))
        enText.append(preprocess_sentence(rowEn['text']))

    return frText, enText
  #lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  #word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  #return zip(*word_pairs)

def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

def load_dataset():
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset()

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [15]:
fr, en = create_dataset()
print(fr[-1])
print(en[-1])

<start> je conviens que nous avons besoin d un agenda social ambitieux qui englobera la lutte contre la pauvrete et l exclusion sociale . <end>
<start> i agree that we need an ambitious social agenda which will include combating poverty and social exclusion <end>


In [0]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset()

# Calculate max_length of the target tensors
#max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [17]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

8800 8800 2200 2200


In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [19]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
78 ----> when
8 ----> a
6814 ----> senior
1522 ----> defense
2806 ----> expert
700 ----> recently
6495 ----> testified
162 ----> before
8 ----> a
61 ----> us
6991 ----> congressional
46 ----> commission
16 ----> on
248 ----> china
21 ----> s
511 ----> military
4288 ----> capability
63 ----> he
1407 ----> detailed
1 ----> the
6992 ----> extraordinarily
2437 ----> robust
825 ----> weapons
1900 ----> program
1 ----> the
69 ----> people
21 ----> s
4449 ----> liberation
1815 ----> army
5574 ----> pla
27 ----> has
51 ----> been
2763 ----> pursuing
3 ----> <end>

Target Language; index to word mapping
2 ----> <start>
143 ----> alors
31 ----> qu
20 ----> il
13419 ----> temoignait
453 ----> devant
6 ----> la
46 ----> commission
834 ----> parlementaire
1006 ----> americaine
30 ----> sur
11 ----> les
1301 ----> capacites
1406 ----> militaires
3228 ----> chinoises
5 ----> ,
17 ----> un
4768 ----> expert
908 ----> militaire
13420 ----> detaillai

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [21]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 94]), TensorShape([64, 113]))

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [23]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 94, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [25]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 94, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [27]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 16267)


In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)


    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [0]:
EPOCHS = 10


for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [44]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
max_length_targ = 40
max_length_inp = 1000

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  #plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [47]:
translate("i agree that we need an ambitious social agenda which will include combating poverty and social exclusion")

Input: <start> i agree that we need an ambitious social agenda which will include combating poverty and social exclusion <end>
Predicted translation: impulsions juridiques oiseaux debat disproportionne surevaluer fabriquer dut honneur modeste accueillis alcoolisees seneque legitimer intitulee commission taraudent geree apprecierions endroit natura permanent emettrions risques salariaux ouest triomphe innocenter herman norme interpellent regulation traces declarant coupes evidence expurgees connection ouest triomphe 


In [0]:
#print(valid_lang1_en['text'][8802])
'''
myEn = input_tensor_val[10]
myFr = target_tensor_val[10]
enSentence = ""
frSentence = " "


for word in myEn:
  if word != 0 and word != 2 and word != 3:
    enSentence += inp_lang.index_word[word] + " "

for word in myFr:
  if word != 0 and word != 2 and word != 3:
    frSentence += targ_lang.index_word[word] + " "


print(enSentence)
print(frSentence)
'''

# Create text file with validation sentences
transl = ""
valid_En_To_Translate = []
for sentence in input_tensor_val:
  for word in sentence:
    if word != 0 and word != 2 and word != 3:
      transl += inp_lang.index_word[word] + " "
  valid_En_To_Translate.append(transl)
  transl = ""


with open(r"valid_en_dataset.txt","a+") as file:
  for sentence in valid_En_To_Translate:
    file.write('%s\n' % sentence)

file.close()

In [0]:
validTarget = ""
valid_TargetList = []
for sentence in target_tensor_val:
  for word in sentence:
    if word != 0 and word != 2 and word != 3:
      validTarget += targ_lang.index_word[word] + " "
  valid_TargetList.append(validTarget)
  validTarget = ""

with open(r"valid_Target.txt","a+") as file:
  for sentence in valid_TargetList:
    file.write('%s\n' % sentence)

In [57]:
with open("valid_en_dataset.txt", 'r') as f:
    validText = [line.rstrip() for line in f] # Remove the \n
    f.close()

print(validText)

['the german presidency has also mentioned such a possibility', 'commissioner byrne thank you for your factual explanation but still i have to say that i wonder what it is we are actually doing exactly', 'after that the time comes for them to decide', 'that is the simple message of the olympic truce the olympic truce is not a magic wand for resolving wars it is a window of opportunity', 'during the debate and during the preparations i had a sentence running through my head which has been said repeatedly by the german environment minister norbert rottgen we have already picked the low hanging fruit when it comes to fossil fuels', 'there has not been a decade of television without a definitive dominant tv mom', 'the police found a dead body in an abandoned car near the park', 'in fact i know the industry will hate me for saying this but the logical consequence of the boeing mcdonnell douglas merger is that the next step forward should be a link between airbus and lockheed martin', 'but r

In [65]:
with open("valid_predictions.txt", 'r') as f:
    validPrediction = [line.rstrip() for line in f] # Remove the \n
    f.close()

print(validPrediction)

['le projet d une autre point de nombreuses propositions de la presidence allemande .', 'monsieur le commissaire byrne , je me permets de vous affirmer que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des difficultes que nous sommes des', 'apres tout le temps vient d effectuer un changement .', 'c est la seule facon ii .', 'au cours de la tete de la famille et norbert rottgen , et ensuite .', 'il n y a pas une decennie de television .', 'la police a cause .', 'en tant que penser que les faits , mais je crois que la 

In [71]:
import sacrebleu

bleu_scores = []
for i in range(len(validText)):
    bleu_scores += [sacrebleu.corpus_bleu(validText[i], validPrediction[i]).score]
    
np.mean(bleu_scores)

1.4486100506371546